Amigocloud api exploration
Written by Vivek Bansal
Started Monday July 20th, 2016

This notebook is a way of querying the amigocloud api and datasets and taking that data and putting it into a pandas data frame.
#TODO - convert geom field to create a GeoJson or Shapefile, or possibly geopandas so data can be reuploaded to amigocloud properly.

Based off of https://github.com/amigocloud/amigocloud_samples

In [1]:
from pprint import pprint
from amigocloud import AmigoCloud
import pandas as pd

In [2]:
#Read Only Token
token = ''

In [3]:
ac = AmigoCloud(token)
me = ac.get('me')
me['id']
projects = ac.get(me['visible_projects'])

In [4]:
#Print all Project numbers and datasets.
print 'Datasets in project:'
for each in projects['results']:
    print '#Project# ', each['name'], '#', each['id']
    for result in ac.get(each['datasets'])['results']:
        print '*', result['name'], '*', result['id']
#     for result in ac.get(projects['results'][1]['datasets'])['results']:
#         print '*', result['name']

Datasets in project:
#Project#  VTA Data # 661
* VTA events * 20770
* VTA real-time (Current) * 12556
* Bus Prediction * 20677
* User Feedback * 20756
* April_2016 * 70978
* April2016 * 70977
* January_2016 * 70606
* July_2015_OSM * 68970
* 522 4.5 hour x, y realtime * 29203
* Route 522 2 day 10/30/15 * 29104
* Last hour history * 13858
* Stops October 2014 * 8719
* SELECT * FROM dataset_12556
WHERE timestamp >= extract(epoch FROM (now() - interval '1 day')) and ob * 14067
* SELECT * FROM dataset_12556
WHERE object_id = '2309 - 920956' and timestamp >= extract(epoch FROM (n * 14066
* Trapeze Streets * 4247
* tracking * 9892
* July2015 * 24685
* July_2015 * 24684
* Bike Lanes and Routes for OSM * 18024
* VTA Bounding Box * 22492
#Project#  Bus Stop Database Update # 3098
* busstopinv_update * 27816
* busstopinv_update (copy) * 75192
* 147701geom * 75087
* busstopinv * 27815
* stop147701 * 74330
* tracking * 24460
* LRT_Stops_RTIID * 30859
* Light_Rail_Stops * 26536
* shelters * 26091
* 

In [5]:
#busstopinv_update
PROJECT_OWNER = me['id']
PROJECT_ID = 3098
DATASET_ID = 27816


In [6]:
dataset_url = '/users/{user_id}/projects/{project_id}/datasets/{dataset_id}'
dataset = ac.get(dataset_url.format(user_id=PROJECT_OWNER,
                                    project_id=PROJECT_ID,
                                    dataset_id=DATASET_ID))
table_name = dataset['table_name']

print 'The table name we have to use in SQL queries is', table_name
print 'The table has', dataset['feature_count'], 'rows'

The table name we have to use in SQL queries is dataset_27816
The table has 3978 rows


In [7]:
# AmigoCloud supports SQL operators of PostgreSQL and PostGIS. For more
# information please refer to:
# http://postgis.net/documentation/ and
# http://postgis.net/docs/manual-2.1/reference.html

sql_url = '/users/{user_id}/projects/{project_id}/sql'.format(
    user_id=PROJECT_OWNER, project_id=PROJECT_ID
)

In [8]:
query = 'SELECT * FROM {table}'.format(table=table_name)
total_rows = dataset['feature_count']
offset = 0
limit = 100
rows = []
response = ac.get(dataset['master'])
master = response['master']

In [9]:
#while len(rows) < total_rows:
while len(rows) < total_rows:
    response = ac.get(sql_url, {'query': query, 'offset': offset,
                                'limit': limit, 'state': master,
                                'dataset_id': DATASET_ID})

    if not offset:  # i.e. If first request
        print 'The schema of the result is:'
        pprint(response['columns'])

    fetched_rows = len(response['data'])
    offset += fetched_rows
    rows += response['data']
    # print 'Fetched', fetched_rows, 'rows'

# print 'These are all the rows:'
#pprint(rows)

The schema of the result is:
[{u'name': u'ogc_fid', u'type': u'integer'},
 {u'name': u'lat', u'type': u'string'},
 {u'name': u'long', u'type': u'string'},
 {u'name': u'feature', u'type': u'string'},
 {u'name': u'rtiid', u'type': u'string'},
 {u'name': u'stop', u'type': u'string'},
 {u'name': u'route_number', u'type': u'string'},
 {u'name': u'routes_listed', u'type': u'string'},
 {u'name': u'rti', u'type': u'string'},
 {u'name': u'standard_vta_sign', u'type': u'string'},
 {u'name': u'two_sided_sign', u'type': u'string'},
 {u'name': u'other_agencies_signs', u'type': u'string'},
 {u'name': u'rti_decal', u'type': u'string'},
 {u'name': u'others_using_stop', u'type': u'string'},
 {u'name': u'adopt_a_stop', u'type': u'string'},
 {u'name': u'information_sign', u'type': u'string'},
 {u'name': u'other_info_sign', u'type': u'string'},
 {u'name': u'tactile_signs', u'type': u'string'},
 {u'name': u'pole', u'type': u'string'},
 {u'name': u'other_pole', u'type': u'string'},
 {u'name': u'vta_bench', 

In [10]:
len(rows)

3978

In [11]:
response = ac.get(sql_url, {'query': query, 'offset': offset,'limit': limit, 
                            'state': master,'dataset_id': DATASET_ID})


In [12]:
df = pd.DataFrame()

In [13]:
df = pd.DataFrame()
inv_update= df.from_dict(rows,orient='columns')

In [14]:
inv_update.head()

,ac_asphalt_paving,accessible_path_to_crossing,ad_bench,ada_accessible,adjacent_property,adopt_a_stop,amigo_id,bench_pad,boarding_area,boarding_pavement,...,sidewalk_width,simmie_seat,standard_vta_sign,stop,tactile_signs,tc_other,tc_with_shelter,two_sided_sign,vta__trash_container,vta_bench
0,Not Applicable,Yes,0,No,Commercial,No,768451c9906e4567ba56a340ab444cb7,Yes,Yes,PCC Concrete,...,10,None,Yes,418005,None,None,No,No,1,1
1,Not Applicable,Yes,0,Yes,Residential,No,d6aa43f47a3f498ab8083e31d39b0ff4,Yes,Yes,PCC Concrete,...,5,None,Yes,416809,Bus,None,Not Applicable,No,0,1
2,Not Applicable,None,0,None,Residential,No,dc0c52b4979f408e8737f4b0d0093421,Yes,Yes,PCC Concrete,...,6,None,Yes,416808,None,None,None,No,0,0
3,Not Applicable,Yes,0,Yes,Residential,Yes,f691671d4884486790cf81a332bba291,No,Yes,Gravel,...,6,None,Yes,416806,None,None,Not Applicable,No,0,0
4,Not Applicable,Yes,0,Yes,Residential,Yes,466ed668e5b349b696ddc1dfd9312d9d,Yes,Yes,Gravel,...,6,None,Yes,406817,Bus,None,Not Applicable,Yes,0,1


In [15]:
#TODO - Convert GEOMS which are WKT hexadecimal string or WKB to some other format
inv_update['geom'][0]

u'0101000020E61000008CF337A110785EC08BFDA713B7B54240'

In [16]:
#find column differences between data frames
#missing_stop_with_geo.columns.difference(inv_update.columns)